In [ ]:
# Installer la bibliothèque Kaggle
!pip install -q kaggle

# Téléverser votre fichier kaggle.json
from google.colab import files
files.upload()
!mv "kaggle (1).json" kaggle.json

# Configurer Kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Télécharger le dataset "Road Sign Detection"
!kaggle datasets download -d andrewmvd/road-sign-detection

# Décompresser le fichier téléchargé
!mkdir road_sign_detection
!unzip road-sign-detection.zip -d road_sign_detection


In [ ]:
!pip install torch torchvision torchaudio transformers pycocotools opencv-python


In [ ]:
!pip install -q ultralytics tensorboard

In [ ]:
import torch as torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import albumentations as A
import torchaudio
from pycocotools.coco import COCO
import cv2
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, T5TokenizerFast, T5ForConditionalGeneration
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
root_dir = '/content/road_sign_detection'
images_dir = os.path.join(root_dir, 'images')
annotations_dir = os.path.join(root_dir, 'annotations')
train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')
test_dir = os.path.join(root_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'annotations'), exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'annotations'), exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(os.path.join(test_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'annotations'), exist_ok=True)
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.png') or f.endswith('.jpg')])
annotation_files = sorted([f for f in os.listdir(annotations_dir) if f.endswith('.xml')])
train_images, test_images, train_annotations, test_annotations = train_test_split(
    image_files, annotation_files, test_size=0.2, random_state=42, shuffle=False
)

val_images, test_images, val_annotations, test_annotations = train_test_split(
    test_images, test_annotations, test_size=0.5, random_state=42, shuffle=False
)

def move_files(file_list, source_dir, dest_dir):
    for file_name in file_list:
        shutil.move(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

move_files(train_images, images_dir, os.path.join(train_dir, 'images'))
move_files(train_annotations, annotations_dir, os.path.join(train_dir, 'annotations'))

move_files(val_images, images_dir, os.path.join(val_dir, 'images'))
move_files(val_annotations, annotations_dir, os.path.join(val_dir, 'annotations'))

move_files(test_images, images_dir, os.path.join(test_dir, 'images'))
move_files(test_annotations, annotations_dir, os.path.join(test_dir, 'annotations'))

# Définir les noms de classe
class_names = ['speedlimit', 'stop', 'crosswalk', 'trafficlight']

# Créer le fichier de configuration des données
data_yaml = """
train: /content/road_sign_detection/train/images
val: /content/road_sign_detection/val/images

nc: 4  # nombre de classes
names: ['speedlimit', 'stop', 'crosswalk', 'trafficlight']
"""

with open('/content/data.yaml', 'w') as f:
    f.write(data_yaml)

# Charger un modèle pré-entraîné YOLOv5s
model = YOLO('yolov5s.pt')
results = model.train(data='/content/data.yaml', epochs=50, imgsz=640, batch=32, project='runs/train', name='exp', exist_ok=True)
results = model.val(data='/content/data.yaml')
model.save('/content/road_sign_detection/best_model.pt')


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/train


In [ ]:
!pip install gtts


In [ ]:
import os
from ultralytics import YOLO
from matplotlib import pyplot as plt
import cv2
from gtts import gTTS
import IPython.display as ipd

model = YOLO('/content/road_sign_detection/best_model.pt')
test_images_dir = '/content/road_sign_detection/test/images'
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.png') or img.endswith('.jpg')]
def generate_description(results):
    descriptions = []
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls)
            score = float(box.conf)
            descriptions.append(f"Detected {model.names[class_id]} with confidence {score:.2f}")
    return " ".join(descriptions)
def text_to_speech(text):
    if text:
        tts = gTTS(text=text, lang='en')
        tts.save("output.mp3")
        ipd.display(ipd.Audio("output.mp3"))
    else:
        print("No detections to speak about.")
for img_path in test_images:
    results = model(img_path)
    description = generate_description(results)
    print(description)
    text_to_speech(description)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    for result in results:
        for box in result.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
    plt.imshow(img)
    plt.title(description)
    plt.show()


In [ ]:
from ultralytics import YOLO
import os
from matplotlib import pyplot as plt
import cv2

model = YOLO('/content/road_sign_detection/best_model.pt')
test_images_dir = '/content/road_sign_detection/test/images'
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.png') or img.endswith('.jpg')]
for img_path in test_images:
    results = model(img_path)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    for result in results:
        for box in result.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
    plt.imshow(img)
    plt.show()
